In [97]:
# https://www.geeksforgeeks.org/python-sqlite-creating-a-new-database/#

from sqlalchemy import create_engine

from sqlalchemy.orm import declarative_base, Session, relationship

from sqlalchemy import Column, Integer, String, Float, DateTime, ForeignKey

from sqlalchemy.schema import ForeignKeyConstraint

import sqlite3 

import pandas as pd

from datetime import datetime

In [98]:
df = pd.read_csv("Exports/cleaned.csv")
df.head()

,ISO3,ISO3.1,country,mkt_name,DATES,currency,data_coverage,data_coverage_recent,index_confidence_score,spatially_interpolated,...,inflation_yam,trust_yam,o_yogurt,c_yogurt,inflation_yogurt,trust_yogurt,o_food_price_index,c_food_price_index,inflation_food_price_index,trust_food_price_index
0,0,AFG,Afghanistan,Badakhshan,2007-01-01,AFN,34.73,82.29,0.93,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.61,0.61,NaN,7.7
1,1,AFG,Afghanistan,Badakhshan,2007-02-01,AFN,34.73,82.29,0.93,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.61,0.60,NaN,7.7
2,2,AFG,Afghanistan,Badakhshan,2007-03-01,AFN,34.73,82.29,0.93,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.60,0.61,NaN,7.7
3,3,AFG,Afghanistan,Badakhshan,2007-04-01,AFN,34.73,82.29,0.93,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.61,0.63,NaN,7.7
4,4,AFG,Afghanistan,Badakhshan,2007-05-01,AFN,34.73,82.29,0.93,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.64,0.69,NaN,7.7


In [99]:
market_df = df[["country", "mkt_name", "ISO3.1", "currency"]]
market_df.drop_duplicates(inplace=True)
market_df.rename(columns={"mkt_name": "market", "ISO3.1": "code"}, inplace=True)
market_df

C:\Users\Gam3sTr\AppData\Local\Temp\ipykernel_8208\3562340101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_df.drop_duplicates(inplace=True)
C:\Users\Gam3sTr\AppData\Local\Temp\ipykernel_8208\3562340101.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_df.rename(columns={"mkt_name": "market", "ISO3.1": "code"}, inplace=True)


,country,market,code,currency
0,Afghanistan,Badakhshan,AFG,AFN
219,Afghanistan,Badghis,AFG,AFN
438,Afghanistan,Baghlan,AFG,AFN
657,Afghanistan,Balkh,AFG,AFN
876,Afghanistan,Bamyan,AFG,AFN
...,...,...,...,...
501465,"Yemen, Rep.",Sayoun City,YEM,YER
501660,"Yemen, Rep.",Soqatra (Hudaibo),YEM,YER
501855,"Yemen, Rep.",Taiz City,YEM,YER
502050,"Yemen, Rep.",Zungubar City,YEM,YER


In [100]:
market_df.columns

Index(['country', 'market', 'code', 'currency'], dtype='object')

In [101]:
food_prices_db = "food_prices.sqlite"
  
try: 
  conn = sqlite3.connect(food_prices_db) 
  print(f"Database {food_prices_db} formed.") 
except: 
  print(f"Database {food_prices_db} not formed.")

Database food_prices.sqlite formed.


In [102]:
Base = declarative_base()

In [103]:
class Market(Base):
    __tablename__ = 'markets'
    country = Column(String(50), primary_key=True)
    market_name = Column(String(50), primary_key=True)
    code = Column(String(3))
    currency = Column(String(3))
    food_products = relationship("Food", back_populates="market")

In [ ]:
class Food(Base):
    __tablename__ = 'food_prices'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(20))
    date = Column(DateTime)
    open = Column(Float)
    close = Column(Float)
    inflation = Column(Float)
    trust = Column(Float)
    country = Column(String(50))
    market_name = Column(String(50))
    
    __table_args__ = (
        ForeignKeyConstraint(
            ['country', 'market_name'],
            ['markets.country', 'markets.market_name']
        ),
    )
    
    market = relationship("Market", back_populates="food_products")

In [105]:
# food_price = Food(
#     name = "Bread",
#     date = datetime.strptime("2017-01-01", "%Y-%m-%d").date(),
#     year = 2017,
#     month = 1,
#     open = 32.43,
#     high = 32.65,
#     low = 32.03,
#     close = 32.4,
#     inflation = 29.37,
#     trust = 7.4,
#     market = market
# )

In [106]:
engine = create_engine("sqlite:///food_prices.sqlite")
conn = engine.connect()

In [107]:
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [108]:
for index, row in market_df.iterrows():
    market = Market(
        country= row["country"],
        market_name = row["market"], 
        code= row["code"],
        currency = row["currency"]
    )
    session.add(market)

In [ ]:

open_food=[col for col in df.columns if col.startswith(('o_'))]
close_food=[col for col in df.columns if col.startswith(('c_'))]
trust_food=[col for col in df.columns if col.startswith(('trust_'))]
inflation_food=[col for col in df.columns if col.startswith(('inflation_'))]
for index, row in df.iterrows():
    food = Market(
        name = row["?"],
        date = row["dates"],
        open = row["?"], 
        close = row["?"],
        inflation = row["?"],
        trust = row["?"],
        country = row["country"],
        market_name = row["mkt_name"],
    )
    session.add(food)

['o_apples', 'c_apples', 'inflation_apples', 'trust_apples', 'o_bananas', 'c_bananas', 'inflation_bananas', 'trust_bananas', 'o_beans', 'c_beans', 'inflation_beans', 'trust_beans', 'o_bread', 'c_bread', 'inflation_bread', 'trust_bread', 'o_bulgur', 'c_bulgur', 'inflation_bulgur', 'trust_bulgur', 'o_cabbage', 'c_cabbage', 'inflation_cabbage', 'trust_cabbage', 'o_carrots', 'c_carrots', 'inflation_carrots', 'trust_carrots', 'o_cassava', 'c_cassava', 'inflation_cassava', 'trust_cassava', 'o_cassava_flour', 'c_cassava_flour', 'inflation_cassava_flour', 'trust_cassava_flour', 'o_cassava_meal', 'c_cassava_meal', 'inflation_cassava_meal', 'trust_cassava_meal', 'o_cheese', 'c_cheese', 'inflation_cheese', 'trust_cheese', 'o_chickpeas', 'c_chickpeas', 'inflation_chickpeas', 'trust_chickpeas', 'o_chili', 'c_chili', 'inflation_chili', 'trust_chili', 'o_coffee_instant', 'c_coffee_instant', 'inflation_coffee_instant', 'trust_coffee_instant', 'o_couscous', 'c_couscous', 'inflation_couscous', 'trust_co

InvalidRequestError: Implicitly combining column food_prices.o_apples with column food_prices.o_apples under attribute 'o_apples'.  Please configure one or more attributes for these same-named columns explicitly.

In [109]:

session.commit()

In [110]:
market_list = session.query(Market)
for market in market_list:
    print(market.market_name)

Badakhshan
Badghis
Baghlan
Balkh
Bamyan
Daykundi
Farah
Faryab
Fayzabad
Gardez
Ghazni
Ghor
Hilmand
Hirat
Jalalabad
Jawzjan
Kabul
Kandahar
Kapisa
Khost
Kunar
Kunduz
Laghman
Logar
Maidan Wardak
Maymana
Mazar
Nangarhar
Nili
Nimroz
Nuristan
Paktika
Paktya
Panjsher
Parwan
Samangan
Sar-e-Pul
Takhar
Uruzgan
Zabul
Market Average
Armavir
Artashat
Ashtarak
Berd
Gavar
Gyumri
Hrazdan
Ijevan
Kapan
Vagharshapat
Vanadzor
Yeghegnadzor
Yerevan
Market Average
Bandaga
Biyogwa
Biyorwa
Bubanza
Buhiga
Bujumbura
Bukeye
Bukirasazi
Burambi
Bururi
Busoni
Buyengero
Cankuzo
Chez Siyoni
Cibitoke
Gasenyi
Gasorwe
Gatabo
Gihamagara
Giharo
Gihogazi
Gisuru
Gitaza
Gitega
Gitega Centre
Giteranyi
Gitobe
Jene
Kamenge
Kanyosha
Kayanza
Kayero
Kayogoro
Kayongozi
Kigamba
Kinama
Kinyinya
Kirehe
Kirundo
Kivyuka
Kobero
Mabanda
Mabayi
Makamba
Marangara
Masanganzira
Matana
Mihigo
Mishiha
Muhanga
Mukenke
Muramvya
Musenyi
Mutaho
Muyange
Muyinga
Muzinda
Mwakiro
Mwaro
Ngozi
Nyabihanga
Nyabikere
Nyagasasa
Rubaho
Rugombo
Ruhehe
Rumonge
Ru

In [111]:
food_list = session.query(Food)
for food in food_list:
    print(food.name)

In [112]:
Base.metadata.drop_all(engine)
conn.close()